In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from gensim.models import Word2Vec
from keras.callbacks import ModelCheckpoint
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.decomposition import PCA

C:\Users\AAKASH\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\AAKASH\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from nltk.corpus import stopwords
import string

In [3]:
filename = 'paragraph.txt'
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [4]:
all_words = word_tokenize(raw_text)
punc = list(string.punctuation)

In [5]:
punc.remove('.')
punc += ['‘', '’', '--']

In [6]:
pure_words = []
for word in all_words:
    if(word not in punc):
        pure_words.append(word)
text = ' '.join(pure_words)

In [7]:
text

"samuel butler has a lot to answer for . but for him a modern traveler could spend his time peacefully admiring the scenery instead of feeling himself bound to dog the simple and grotesque of the world for the sake of their too-human comments . it is his fault if a peasant 's _naïveté_ has come to outweigh the beauty of rivers and the remarks of clergymen are more than mountains . it is very restful to give up all effort at observing human nature and drawing social and political deductions from trifles and to let oneself relapse into wide-mouthed worship of the wonders of nature . and this is very easy at niagara . niagara means nothing . it is not leading anywhere . it does not result from anything . it throws no light on the effects of protection nor on the facility for divorce in america nor on corruption in public life nor on canadian character nor even on the navy bill . it is merely a great deal of water falling over some cliffs . but it is very remarkably that . the human race a

In [8]:
sentences = sent_tokenize(text)
sentence_words = []
for sentence in sentences:
    words = word_tokenize(sentence)
    sentence_words.append(words)

In [9]:
all_words = word_tokenize(text)
n_words = len(all_words)

In [10]:
model = Word2Vec(sentence_words, min_count = 1)
unique_words = list(model.wv.vocab)
X = model[model.wv.vocab]

C:\Users\AAKASH\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
X[0]

array([ 3.5112223e-03, -1.1111058e-03,  9.3224703e-04,  1.2056561e-03,
        2.7394752e-04, -4.5081587e-03, -5.4597511e-04,  2.5315809e-03,
       -1.0564079e-03, -4.3983051e-05, -1.1389211e-03, -2.2470227e-03,
       -1.8412161e-04,  2.2941332e-03,  4.0915054e-03,  3.7332824e-03,
        8.3137007e-04, -2.3677328e-03, -9.7633027e-05, -1.8823985e-03,
       -4.5288387e-03, -2.4552969e-04, -4.9671303e-03, -4.0496951e-03,
        3.1774545e-03, -4.0811263e-03,  1.2995029e-03,  1.2434681e-03,
        3.3467337e-03,  1.9549283e-03,  1.3234899e-03,  4.3767408e-04,
        2.7604424e-03,  4.2024348e-03,  2.3399284e-03,  1.8831862e-03,
       -1.9913029e-03,  2.0207732e-03, -1.0471378e-03,  4.4320053e-03,
        1.1498063e-03, -3.7819743e-03, -3.5451443e-04, -3.4483885e-03,
        4.3857060e-03, -4.6761392e-04,  5.9742632e-04, -4.6755769e-03,
        1.0845689e-03,  3.8723983e-03,  5.9050263e-04, -3.2788974e-03,
       -2.2682389e-03, -2.7858422e-03,  3.0881655e-03, -3.1968367e-03,
      

In [12]:
k = 100

In [77]:
# pca = PCA(n_components = k)
# result = pca.fit_transform(X)

In [13]:
result = X

In [14]:
word_to_vec = {}
word_to_int = {}
# vec_to_word = {}
int_to_word = {}
int_to_vec = {}
for i in range(len(unique_words)):
    word_to_vec[unique_words[i]] = result[i]
    word_to_int[unique_words[i]] = i
#     vec_to_word[result[i][0]] = unique_words[i]
    int_to_word[i] = unique_words[i]
    int_to_vec[i] = result[i]

In [15]:
seq_length = 20
dataX = []
dataY = []
for i in range(n_words - seq_length):
    seq_in = all_words[i : i + seq_length]
    seq_out = all_words[i + seq_length]
    data_x = [word_to_vec[word] for word in seq_in]
    data_y = [word_to_int[seq_out]]
    dataX.append(data_x)
    dataY.append(data_y)
data_points = len(dataX)

In [16]:
X = np.reshape(dataX, (data_points, seq_length, k))
Y = np_utils.to_categorical(dataY)

In [17]:
model = Sequential()
model.add(LSTM(100, input_shape = (X.shape[1], X.shape[2]), return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(Y.shape[1], activation = 'softmax'))

In [18]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [40]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, Y, epochs = 200, batch_size = 10)

Epoch 1/200
1993/1993 [==============================] - 11s 6ms/step - loss: 0.2076 - acc: 0.9313
Epoch 2/200
1993/1993 [==============================] - 7s 3ms/step - loss: 0.2311 - acc: 0.9247
Epoch 3/200
1993/1993 [==============================] - 7s 4ms/step - loss: 0.2252 - acc: 0.9308
Epoch 4/200
1993/1993 [==============================] - 7s 3ms/step - loss: 0.1677 - acc: 0.9398
Epoch 5/200
1993/1993 [==============================] - 7s 4ms/step - loss: 0.2140 - acc: 0.9288
Epoch 6/200
1993/1993 [==============================] - 7s 4ms/step - loss: 0.1916 - acc: 0.9418
Epoch 7/200
1993/1993 [==============================] - 7s 4ms/step - loss: 0.1955 - acc: 0.9353
Epoch 8/200
1993/1993 [==============================] - 8s 4ms/step - loss: 0.1789 - acc: 0.9428
Epoch 9/200
1993/1993 [==============================] - 7s 3ms/step - loss: 0.1866 - acc: 0.9453
Epoch 10/200
1993/1993 [==============================] - 7s 4ms/step - loss: 0.2008 - acc: 0.9373
Epoch 11/200
1993/

1993/1993 [==============================] - 7s 4ms/step - loss: 0.1876 - acc: 0.9393
Epoch 165/200
1993/1993 [==============================] - 9s 4ms/step - loss: 0.2178 - acc: 0.9333
Epoch 166/200
1993/1993 [==============================] - 7s 4ms/step - loss: 0.1858 - acc: 0.9328
Epoch 167/200
1993/1993 [==============================] - 8s 4ms/step - loss: 0.1905 - acc: 0.9368
Epoch 168/200
1993/1993 [==============================] - 7s 4ms/step - loss: 0.1676 - acc: 0.9453
Epoch 169/200
1993/1993 [==============================] - 8s 4ms/step - loss: 0.2377 - acc: 0.9242
Epoch 170/200
1993/1993 [==============================] - 7s 4ms/step - loss: 0.2316 - acc: 0.9237
Epoch 171/200
1993/1993 [==============================] - 8s 4ms/step - loss: 0.1624 - acc: 0.9493
Epoch 172/200
1993/1993 [==============================] - 7s 3ms/step - loss: 0.1685 - acc: 0.9518
Epoch 173/200
1993/1993 [==============================] - 9s 4ms/step - loss: 0.2174 - acc: 0.9318
Epoch 174/200


In [42]:
start = np.random.randint(0, len(dataX)-1)
# start = 1
pattern = dataX[start]
print("Seed:")
# for i in range(len(pattern)):
#     print(vec_to_word[pattern[i][0]], end = ' ')
print(" : ")

for i in range(300):
    x = np.reshape(pattern, (1, len(pattern), k))
    prediction = model.predict(x, verbose = 0)
    index = np.argmax(prediction)
#     print(index)
    result = int_to_word[index]
    print(result, end = ' ')
    pattern.append(int_to_vec[index])
    pattern = pattern[1 : len(pattern)]
print("\nDone.")

Seed:
 : 
in all comparisons that rise in the heart the river with its multitudinous waves and its single current likens itself to a life whether of an individual or of a community . a man 's life is of many flashing moments and yet one stream a nation 's flows through all its citizens and yet is more than they . in such places one is aware with an almost insupportable and yet comforting certitude that both men and nations are hurried onwards to their ruin or ending as inevitably as this dark flood . some go down to it unreluctant and meet it like the river not without nobility . and as incessant as inevitable and as unavailing as the spray that hangs over the falls is the white cloud of human crying ... . with some such thoughts does the platitudinous heart win from the confusion and thunder of a niagara peace that the quietest plains or most stable hills can never give . it the falls from touts finally . the facility of the thunder . in a mile bound throwing is the stream home and br

the face . farther away in the roar and hissing clouds of spray seem literally to slide down some invisible plane of air . beyond the foot of the falls the river is like a slipping floor of marble green with veins of dirty white made by the scum that was foam . it slides very quietly and slowly down for a mile or two sullenly exhausted . then it turns to a dull sage green and hurries more swiftly smooth and ominous . as the walls of the ravine close in trouble stirs and the waters boil and eddy . these are the lower rapids a sight more terrifying than the falls because less intelligible . close in its bands of rock the river surges tumultuously forward writhing and leaping as if inspired by a demon . it is pressed by the straits into a visibly convex form . great planes of water slide past . sometimes it is thrown up into a pinnacle of foam higher than a house or leaps with incredible speed from the crest of one vast wave to another along the shining curve between like the spring of a wild beast . its motion 